# 文字

## エンコーディング

テキストデータとはそれを構成する文字と文字コードを並べたものである. 

文字を文字コードに変換することをエンコードという. 文字コードを文字に変換することをデコードという. 

文字コードは次のように複数種類ある．



In [3]:
codes = ["EUC-JP", "Shift_JIS", "UTF-8"]
for cd in codes:
    print(list("京".encode(cd)))

[181, 254]
[139, 158]
[228, 186, 172]


エンコードされた文字，つまり数値から文字コードを推定するには次のようにする. 

In [5]:
import chardet

string = "明日，京都へ行きます"
encoded = string.encode("UTF-8")
chardet.detect(encoded)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}

上記があれば次のようなエンコーディングに応じて読み込むの対応が可能なプログラムをかける. 

In [45]:
import chardet

def get_string_from_file(filename):
    with open(filename, "rb") as f:
        d = f.read()
        e = chardet.detect(d)["encoding"]
        if e is None:
            e = "UTF-8"
        return d.decode(e)

## 文字 N グラム

文字Nグラムとは長さNの部分文字列のことである．

通常は単にNグラムといえば，単語のNグラム，つまり単語がN個並んだものを指す．



In [8]:
def get_ngram(string, N=1):
    return [string[i:(i+N)] for i in range(len(string) - N + 1)]

string = "情報検索"
get_ngram(string, 1)


['情', '報', '検', '索']

In [9]:

get_ngram(string, 2)

['情報', '報検', '検索']

文字Nグラムは，索引や検索に使われるほか，文字Nグラムの頻度分布を文章の特徴量として使うことがある. 

In [46]:
from collections import Counter
def get_most_common_ngram(filename, N = 1, k = 1):
    s = get_string_from_file(filename)
    return Counter(get_ngram(s, N)).most_common(k)

In [74]:
from pathlib import Path
data_dir = Path("irpb-files/data")
target_files = list(data_dir.glob("ch01/0[0-9].txt"))


In [48]:
for f in target_files:
    print(get_most_common_ngram(f))


[('祭', 3)]
[('は', 4)]
[('す', 6)]
[('0', 2)]


In [49]:
get_most_common_ngram(data_dir / "ch01" / "melos.txt", N = 3, k = 5)

[('メロス', 76), ('った。', 53), ('ロスは', 47), ('のだ。', 37), ('。メロ', 33)]

文字Nグラムは作者の推定などにも役立つ. 

## 正規表現




In [52]:
import re

string = "やっぱり『つぶ餡』が好き"
pattern = '『.*』'
result = re.search(pattern, string)
result.group(0)

'『つぶ餡』'

groupはグループ化の順番で選択するもの. ０の場合にはマッチした文字列全体.

次のように入れ子になっている場合には，内側に行くほど順番はあと. 

In [59]:
pattern = '『((..).*)』'
result = re.search(pattern, string)
result.group(0)

'『つぶ餡』'

In [60]:
result.groups()

('つぶ餡', 'つぶ')

In [63]:
result.group(2)

'つぶ'

そのほかには，グループの参照と最短一致/最長一致について覚えておくこと．

In [67]:
string = "このぼたもちはとてももちもちしている"
pattern = r"(..)\1"
ret = re.search(pattern, string)
ret.group()

'もちもち'

In [68]:
string = "『つぶ餡』にするか『こし餡』にするか"
pattern = r"『(.*?)』" # 最短一致
ret = re.search(pattern, string)
ret.group()

'『つぶ餡』'

マッチした文字のイテレータ化. 

In [71]:
string = "『つぶ餡』にするか『こし餡』にするか"
pattern = r"『(.*?)』"
ret = re.findall(pattern, string)
ret

['つぶ餡', 'こし餡']

正規表現の使った処理の方法として，例えば「京都」という単語の前後n文字を抽出することを考える. 


In [85]:
import re

def get_snippet_from_file(fielname, query, width = 2):
    s = get_string_from_file(fielname)
    p = '.{0,%d}%s.{0,%d}' % (width, query, width)
    r = re.search(p, s)
    if r :
        return r.group(0)
    else : 
        return None

In [88]:
for f in target_files:
    print(get_snippet_from_file(f, "京都"))

祭は京都三大
None
京都のお
は東京都江東


In [81]:
target_files[0]

WindowsPath('irpb-files/data/ch01/01.txt')